## Train Network to Drive Car in Simulation

## Data Preprocessing

In [1]:
import cv2
import keras
import tensorflow as tf
import csv
import numpy as np
import os 

Using TensorFlow backend.


In [2]:
import os
######
CENTER_IDX = 0
LEFT_IDX = 1
RIGHT_IDX = 2
STEERING_ANGLE_IDX = 3


#### READ INPUT DATA

In [26]:


def extract_file_name(file):
    """
    Assume last backslash
    """
    assert(file is not None)
    name_start = file.rfind("/")
    return file[name_start+1:]


def read_input_dir(folder_name, center_imgs, left_imgs, right_imgs, steering_angles):
    # OPEN data folder to read data
    # load csv file
    if not os.path.exists(folder_name):
        raise Exception('folder %s does not exist' % folder_name)
    with open(folder_name + "/driving_log.csv") as f:
        reader = csv.reader(f)
        for data in reader:
            # extract name 
            left_img = os.path.join(folder_name+"/IMG", extract_file_name(data[LEFT_IDX]))
            right_img = os.path.join(folder_name+"/IMG", extract_file_name(data[RIGHT_IDX]))
            center_img = os.path.join(folder_name+"/IMG", extract_file_name(data[CENTER_IDX]))
            # append data 
            left_imgs.append(left_img)
            right_imgs.append(right_img)
            center_imgs.append(center_img)
            steering_angles.append(float(data[STEERING_ANGLE_IDX]))

        
        

        

### READ ALL DATA FOLDER 

In [27]:
center_imgs = []
left_imgs = []
right_imgs = []
steering_angles = []
# read each data folder
read_input_dir("../collect_data/drive_forward_data", center_imgs, left_imgs, right_imgs, steering_angles)
print (len(center_imgs), len(steering_angles))
read_input_dir("../collect_data/drive_reverse_data", center_imgs, left_imgs, right_imgs, steering_angles)
print (len(center_imgs), len(steering_angles))
read_input_dir("../collect_data/drive_left_data", center_imgs, left_imgs, right_imgs, steering_angles)
print (len(center_imgs), len(steering_angles))
read_input_dir("../collect_data/drive_right_data", center_imgs, left_imgs, right_imgs, steering_angles)
print (len(center_imgs), len(steering_angles))
read_input_dir("../collect_data2/side_drive_data_1", center_imgs, left_imgs, right_imgs, steering_angles)
print (len(center_imgs), len(steering_angles))
read_input_dir("../collect_data2/side_drive_data2", center_imgs, left_imgs, right_imgs, steering_angles)
print (len(center_imgs), len(steering_angles))
read_input_dir("../collect_data3/bridge_data", center_imgs, left_imgs, right_imgs, steering_angles)
print (len(center_imgs), len(steering_angles))
read_input_dir("../collect_data4", center_imgs, left_imgs, right_imgs, steering_angles)
print (len(center_imgs), len(steering_angles))
read_input_dir("../collect_data5", center_imgs, left_imgs, right_imgs, steering_angles)
print (len(center_imgs), len(steering_angles))


2797 2797
5893 5893
6867 6867
8360 8360
9330 9330
10560 10560
11652 11652
13738 13738
15090 15090


In [28]:
# convert to numpy array
import numpy as np
center_imgs = np.array(center_imgs)
left_imgs = np.array(left_imgs)
right_imgs = np.array(right_imgs)
steering_angles = np.array(steering_angles)

In [29]:
import matplotlib.pyplot as plt
%matplotlib inline

### READ RAW IMAGE DATA

In [36]:
raw_inputs = []
raw_labels = []
# CENTEER
for i, image_path in enumerate(center_imgs):
    if (not os.path.exists(image_path)):
        print("Failed %s " % image_path)
    raw_inputs.append(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))
    raw_labels.append(steering_angles[i])
# for i, image_path in enumerate(left_imgs):
#     if (not os.path.exists(image_path)):
#         print("Failed %s " % image_path)
#     raw_inputs.append(cv2.imread(image_path))
#     raw_labels.append(steering_angles[i] - 0.3)
# for i, image_path in enumerate(right_imgs):
#     if (not os.path.exists(image_path)):
#         print("Failed %s " % image_path)
#     raw_inputs.append(cv2.imread(image_path))
#     raw_labels.append(steering_angles[i] + 0.3)

In [37]:
raw_inputs = np.array(raw_inputs)
raw_labels = np.array(raw_labels)
print(raw_inputs.shape)
print(raw_labels.shape)

(15090, 160, 320, 3)
(15090,)


## NETWORK ARCHITECTURE AND TRAINING

In [38]:
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dropout, MaxPooling2D, Dense, Lambda, Cropping2D
from keras.callbacks import ModelCheckpoint

In [39]:
def create_model():
    model = Sequential()
    # normalize input 
    model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
    model.add(Cropping2D(cropping=((70,25),(0,0)))) 
    model.add(Conv2D(24, 5, 5,activation='relu', subsample=(2,2))) # input: 65x320x3, 31x158x3x16
    model.add(Dropout(p=0.2))
    model.add(Conv2D(36, 5, 5,activation='relu',subsample=(2,2))) # input: 31x158x3x32, 13x76x3x32 
    model.add(Dropout(p=0.2))
    model.add(Conv2D(48, 5, 5,activation='relu',subsample=(2,2))) # input: 31x158x3x32, 13x76x3x32 
    model.add(Dropout(p=0.2))
    model.add(Conv2D(64, 3, 3,activation='relu')) # input: 31x158x3x32, 13x76x3x32 
    model.add(Dropout(p=0.2))
    model.add(Conv2D(64, 3, 3,activation='relu')) # input: 31x158x3x32, 13x76x3x32 
    model.add(Dropout(p=0.2))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dropout(p=0.4))
    model.add(Dense(50))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    return model

In [40]:
filepath="best_model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callback_list = [checkpoint]

In [41]:
my_model = create_model()

In [44]:
my_model.fit(raw_inputs, raw_labels, validation_split=0.2, shuffle=True, batch_size=32, callbacks=callback_list, nb_epoch=20)

Train on 12072 samples, validate on 3018 samples
Epoch 1/20
12072/12072 [==============================] - 12s - loss: 0.0331 - acc: 0.5980 - val_loss: 0.1631 - val_acc: 0.3903
Epoch 2/20
12072/12072 [==============================] - 12s - loss: 0.0307 - acc: 0.5994 - val_loss: 0.1592 - val_acc: 0.3827
Epoch 3/20
12072/12072 [==============================] - 12s - loss: 0.0290 - acc: 0.6001 - val_loss: 0.1668 - val_acc: 0.3757
Epoch 4/20
12072/12072 [==============================] - 12s - loss: 0.0274 - acc: 0.5999 - val_loss: 0.1591 - val_acc: 0.3840
Epoch 5/20
12072/12072 [==============================] - 12s - loss: 0.0265 - acc: 0.6006 - val_loss: 0.1612 - val_acc: 0.3880
Epoch 6/20
12072/12072 [==============================] - 12s - loss: 0.0248 - acc: 0.6003 - val_loss: 0.1569 - val_acc: 0.3860
Epoch 7/20
12072/12072 [==============================] - 12s - loss: 0.0236 - acc: 0.6010 - val_loss: 0.1686 - val_acc: 0.3754
Epoch 8/20
12072/12072 [==============================]

In [43]:
my_model.save('my_model.h5')